In [4]:
!pip install tensorflow
!pip install torch

     ---------------------------------------- 0.0/48.7 kB ? eta -:--:--
     ------------------------------- ------ 41.0/48.7 kB 991.0 kB/s eta 0:00:01
     -------------------------------------- 48.7/48.7 kB 836.6 kB/s eta 0:00:00
   ---------------------------------------- 0.0/390.3 MB ? eta -:--:--
   ---------------------------------------- 0.1/390.3 MB 2.0 MB/s eta 0:03:19
   ---------------------------------------- 0.4/390.3 MB 4.4 MB/s eta 0:01:30
   ---------------------------------------- 1.0/390.3 MB 7.7 MB/s eta 0:00:51
   ---------------------------------------- 2.2/390.3 MB 11.4 MB/s eta 0:00:35
   ---------------------------------------- 3.9/390.3 MB 17.6 MB/s eta 0:00:22
   ---------------------------------------- 4.1/390.3 MB 17.5 MB/s eta 0:00:23
   ---------------------------------------- 4.1/390.3 MB 17.5 MB/s eta 0:00:23
   ---------------------------------------- 4.2/390.3 MB 12.8 MB/s eta 0:00:31
   ---------------------------------------- 4.2/390.3 MB 12.8 MB/s e

In [20]:
import numpy as np 
import pandas as pd 
import os
import torch

In [1]:
!pip install -q transformers

In [ ]:
from transformers import pipeline, set_seed

In [37]:
set_seed(42)

model_name = "openai-gpt"

task_pipe = pipeline('sentiment-analysis', model=model_name) # sentiment-analysis, text-generation

Some weights of OpenAIGPTForSequenceClassification were not initialized from the model checkpoint at openai-gpt and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
task_pipe(["I love this menu", "I hate this menu"])

[{'label': 'LABEL_0', 'score': 0.6342501044273376},
 {'label': 'LABEL_0', 'score': 0.6739563941955566}]

In [13]:
sentiment_pipeline = pipeline("sentiment-analysis")
data = ["I love this menu", "I hate this menu"]
sentiment_pipeline(data)

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


[{'label': 'POSITIVE', 'score': 0.9998526573181152},
 {'label': 'NEGATIVE', 'score': 0.9996575117111206}]

In [18]:
sample = ["I am pretty familiar with this menu",
"I am not pretty familiar with this menu"]

sentiment_pipeline(sample)

[{'label': 'POSITIVE', 'score': 0.997536301612854},
 {'label': 'NEGATIVE', 'score': 0.9994070529937744}]

In [19]:
model_name = "openai-gpt"

# sentiment-analysis, text-generation
task_pipe = pipeline('text-generation', model=model_name) 

task_pipe("Hi, I like deep learning", max_length=30, num_return_sequences=5)

[{'generated_text': 'Hi, I like deep learning. are we going to try it? or are you going to start getting nervous and just listen? " \n " okay'},
 {'generated_text': 'Hi, I like deep learning, a lot. but i hate to admit it, but this isn\'t how my life should be. " i started'},
 {'generated_text': 'Hi, I like deep learning, i think i could learn that in bed. " \n he smiled his full, crooked smile that always caused her heart'},
 {'generated_text': 'Hi, I like deep learning. " \n " then come with me. there are a lot of things i \'ll need you to learn. " \n'},
 {'generated_text': 'Hi, I like deep learning about the things that matter, " i said, " but it\'s not the time for that. " \n he glanced'}]

***BERT***

In [24]:
from transformers import BertModel, BertTokenizer

# Load pre-trained BERT model and tokenizer
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

/Users/kiarash/anaconda3/lib/python3.11/site-packages/transformers/utils/generic.py:260: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(


In [26]:
text = "Hello, how are you?"
inputs = tokenizer(text, return_tensors="pt")
inputs

{'input_ids': tensor([[ 101, 7592, 1010, 2129, 2024, 2017, 1029,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1]])}

In [30]:
outputs = model(**inputs)

token_embeddings = outputs.last_hidden_state  # Embeddings for each token
sentence_embedding = outputs.pooler_output    # Embedding for the full sentence

token_embeddings, sentence_embedding

(tensor([[[-0.0824,  0.0667, -0.2880,  ..., -0.3566,  0.1960,  0.5381],
          [ 0.0310, -0.1448,  0.0952,  ..., -0.1560,  1.0151,  0.0947],
          [-0.8935,  0.3240,  0.4184,  ..., -0.5498,  0.2853,  0.1149],
          ...,
          [-0.2812, -0.8531,  0.6912,  ..., -0.5051,  0.4716, -0.6854],
          [-0.4430, -0.7820, -0.8055,  ...,  0.1949,  0.1081,  0.0130],
          [ 0.5570, -0.1080, -0.2412,  ...,  0.2817, -0.3996, -0.1882]]],
        grad_fn=<NativeLayerNormBackward0>),
 tensor([[-0.9397, -0.4081, -0.9024,  0.8667,  0.6076, -0.1782,  0.9319,  0.2685,
          -0.7918, -1.0000, -0.4899,  0.9625,  0.9823,  0.6102,  0.9614, -0.8728,
          -0.6449, -0.6543,  0.3102, -0.6648,  0.7556,  1.0000,  0.0778,  0.3350,
           0.5094,  0.9948, -0.8847,  0.9590,  0.9761,  0.7384, -0.7787,  0.1232,
          -0.9912, -0.2119, -0.9225, -0.9931,  0.3767, -0.8050, -0.0945,  0.0497,
          -0.9269,  0.2934,  1.0000,  0.0954,  0.4904, -0.3206, -1.0000,  0.2759,
          -0.9

In [34]:
len(token_embeddings[0]), len(sentence_embedding[0])

(8, 768)

***GPT for Embedding***

In [35]:
from transformers import OpenAIGPTModel, OpenAIGPTTokenizer

# Load pre-trained GPT-1 model and tokenizer (use openai-gpt as an equivalent)
model_name = 'openai-gpt'
tokenizer = OpenAIGPTTokenizer.from_pretrained(model_name)
model = OpenAIGPTModel.from_pretrained(model_name)

# Encode input text
text = "Hello, how are you?"
inputs = tokenizer(text, return_tensors="pt")

# Run model and get embeddings
outputs = model(**inputs)
token_embeddings = outputs.last_hidden_state
sentence_embedding = token_embeddings.mean(dim=1)  # Averaged token embeddings

print(sentence_embedding)  # Full sentence embedding

ftfy or spacy is not installed using BERT BasicTokenizer instead of SpaCy & ftfy.


tensor([[ 2.8174e-01,  2.7592e-03,  5.6021e-01, -3.4376e-01,  7.3281e-01,
         -1.1146e-01,  3.2819e-02,  6.0725e-01, -8.7436e-01, -2.8564e-01,
          1.4935e-01, -3.1109e-01,  3.0455e-01,  1.2281e-01, -3.9340e-01,
          7.9554e-01, -7.5981e-02,  1.3187e-01,  3.5532e-01, -1.7587e-01,
         -2.1624e-01,  1.1073e+00,  1.8157e-01, -1.5700e-01, -1.1156e-01,
         -1.6902e+00, -7.6925e-01, -2.5179e-01, -1.2561e-02,  3.9508e-02,
          1.0113e-01, -1.4286e-01,  3.8541e-03,  5.5437e-01,  3.0308e-02,
          1.6165e-01,  5.5464e-01,  7.8183e-01, -4.2568e-01, -1.1689e+00,
          1.0758e+00, -8.1389e-01, -1.6389e-02,  5.0320e-01,  3.8949e-01,
         -8.4439e-02, -7.1579e-01, -6.8663e-03,  7.8756e-02, -5.8644e-01,
         -3.3344e-01,  3.6316e-01, -2.3366e-01,  1.8631e-01,  6.0334e-01,
          3.0710e-01,  1.0519e-01,  1.9623e-02, -2.5148e-01,  3.6555e-01,
         -6.8922e-01,  1.3586e-01, -1.0053e-01, -5.9627e-01, -1.1347e-01,
          2.3653e-02, -6.8185e-01,  6.

In [38]:
len(token_embeddings[0]), len(sentence_embedding[0])

(6, 768)